In [186]:
from __future__ import absolute_import, division, print_function
import torch
from torch.distributions import constraints
from torch.nn import Parameter
import pyro
from pyro.contrib.gp.util import Parameterized
import pyro.distributions as dist
import pyro.infer as infer
import pyro.optim as optim
from pyro.params import param_with_module_name
import pyro.contrib.gp as gp
import pyro.distributions as dist
import matplotlib.pyplot as plt
pyro.clear_param_store()
import tensorflow as tf
import numpy as np 
import pandas as pd
import numpy
import math
import cProfile
import time
torch.manual_seed(101)
pyro.set_rng_seed(1)
import GPy
import numpy as np
from matplotlib import pyplot as plt

In [187]:
# Load the Dataset
y = torch.stack([dist.Normal(4.8, 0.1).sample((150,)),dist.Normal(3.2, 0.3).sample((150,)),dist.Normal(1.5, 0.4).sample((150,)),
 dist.Exponential(0.5).sample((150,))])
# Fixed Inducing Inputs for Sparse Model
X_loc = torch.zeros(150, 2)
# Kenel
kernel = gp.kernels.RBF(input_dim=2,lengthscale=torch.ones(1))
Xu = torch.zeros(20, 2)
# Wrap Sparse GP Model below into GPLVM
gpmodel = gp.models.SparseGPRegression(X_loc, y, kernel, Xu, noise=torch.tensor(1.))
gplvm = gp.models.GPLVM(gpmodel)
# Optimize Bayesian GPLVM
losses = gplvm.optimize(num_steps=2000)

In [188]:
# Get Variational Parameters X_loc and X_scale_tril from VI approximation
X = gplvm.get_param("X_loc")
X = X.detach().numpy()
X_var = gplvm.get_param("X_scale_tril")
X_var = X_var.detach().numpy()

### Matrix Samples from the Variational Distribution

In [189]:
S=500
VS = [] # Vector Containing Matrix Samples (Xs)
for i in range(Samples):
    VS.append(pyro.sample('Variational Distibtuion',dist.MultivariateNormal(gplvm.get_param("X_loc"), scale_tril=gplvm.get_param("X_scale_tril")).independent(gplvm.get_param("X_loc").dim()-1),))


## Bayesian GPLVM

In [190]:
Y = numpy.array(y)
Y = Y.T # Reshaping vector 150 x 4 for Bayesian GPLVM
kernel = GPy.kern.RBF(2, ARD=True,variance=1.0,lengthscale=1.0)

In [191]:
m = GPy.models.BayesianGPLVM(Y, 2,  init="PCA",kernel=kernel,num_inducing=20)
m.data_labels = Y.argmax(axis=1)
loss = m.optimize('scg', messages=1,max_iters=2000)

Running Scaled Conjugate Gradients Code:
  runtime   i      f              |g|        
    00s07  0006   1.213564e+03   1.575275e+04 
    00s16  0013   1.091467e+03   8.581268e+03 
    00s19  0016   8.793471e+02   5.741800e+03 
    02s20  0261   4.674008e+02   5.849822e+01 

 /home/aneeqr/.local/lib/python3.6/site-packages/GPy/kern/src/stationary.py:165: RuntimeWarning:overflow encountered in true_divide
 /home/aneeqr/.local/lib/python3.6/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:invalid value encountered in add


    03s21  0382   4.650911e+02   3.592636e+01 

 /home/aneeqr/.local/lib/python3.6/site-packages/GPy/kern/src/stationary.py:136: RuntimeWarning:overflow encountered in square



    11s23  1330   4.061977e+02   1.327581e+02 
    12s34  1455   4.046699e+02   1.885049e+01 
Runtime:     12s34
Optimization status: converged - relative stepsize



## Evaluating Joing Prob Function

$$
p(\mathbf{Y}|\mathbf{X}) = \prod_{d=1}^D \mathcal{N} (\mathbf{y}_d| \mathbf{0}, \mathbf{K}_{NN}+\beta^{-1}\mathbf{I}_N),
$$
$$
\mathcal{N} (\mathbf{y}_d| \mathbf{0}, \mathbf{K}_{NN}+\beta^{-1}\mathbf{I}_N) = \frac{1}{(2\pi)^{N/2}}\frac{1}{|\mathbf{K}_{NN}+\beta^{-1}\mathbf{I}_N|^{1/2}}\exp\bigg[-\frac{1}{2}\mathbf{y}_d^{\top}\Big(\mathbf{K}_{NN}+\beta^{-1}\mathbf{I}_N\Big)^{-1}\mathbf{y}_d\bigg]
$$
$$
\log p(\mathbf{Y},\mathbf{X}=\mathbf{X}_s) =-\frac{ND}{2}\log 2\pi-\frac{D}{2}\log\Big(|\mathbf{K}_{NN}+\beta^{-1}\mathbf{I}_N|\Big)-\frac{1}{2}\operatorname{trace}\Big(\mathbf{Y}^{\top}\mathbf{P}_{NN}\mathbf{Y}\Big)-\frac{NQ}{2}\log 2\pi-\frac{1}{2}\operatorname{trace}(\mathbf{X}_s\mathbf{X}_s^{\top}),
$$

## Function 

In [192]:
def Joint_Prob(Y,Xs,N,D,Q,kernel,noise_var=0.05):
    K= kernel.K(np.array(Xs),np.array(Xs)) # Covariance matrix for K(Xs,Xs), KNN
    Noise=np.eye(K.shape[0])*np.sqrt(noise_var)  # B^-1 Noise vector
    Noise_term = np.matmul(np.linalg.inv(Noise),np.eye(K.shape[0]))  # B^-1 I
    K_new = np.add(K,Noise_term) #  KNN + B^(-1)I_{N}
    K_inv = np.linalg.inv(K_new) # PNN inverse of KNN
    Prob=1
    Prob = -N*D*0.5*np.log(2*np.pi) - D/2*np.log(np.linalg.det(K_new))-1/2*np.trace(np.matmul(np.matmul(Y.T,K_inv),Y)) - (N*Q/2)*np.log(2*np.pi)-0.5*np.trace(np.matmul(Xs,Xs.T))
    return Prob 

In [193]:
N= y.shape[0]
Q = 2
Join = []
for sample in VS:
    Join.append(Joint_Prob(Y,sample.detach().numpy(),N=150,D=4,Q=2,kernel=kernel,noise_var=0.05))

# Evaluating q(X)

$$
q(\mathbf{x})= \frac{1}{\sqrt { (2\pi)^n|\boldsymbol \Sigma| } }  \exp\left(-{1 \over 2} (\mathbf{x})^{\rm T} \boldsymbol\Sigma^{-1} ({\mathbf x})\right)
$$

$$
\boldsymbol\Sigma_{S} = (\mathbf{K}_{S}+\beta^{-1}\mathbf{I}_S\Big)
$$


$$
\log q(\mathbf{X} = \mathbf{X_{S}}) = -\frac{ND}{2}\log 2\pi-\frac{D}{2}\log\Big(|\boldsymbol \Sigma_{s} |\Big)-\frac{1}{2}\operatorname{trace}\Big(\mathbf{X_{S}}^{\top}\boldsymbol\Sigma^{-1}_{S}\mathbf{X_{S}}\Big)
$$

In [194]:
def Var_Dist(gplvm,Sample,D,noise_var=0.05):
    # Q = K(Xs,Xu)k(Xu,Xu)^-1
    Q = np.matmul(gplvm.base_model.kernel(Sample,gplvm.base_model.Xu).detach().numpy(),np.linalg.inv(gplvm.base_model.kernel(gplvm.base_model.Xu,gplvm.base_model.Xu).detach().numpy()))
     # Q = K(Xs,Xu)k(Xu,Xu)^-1 k(Xs,Xu)
    Q = np.matmul(Q,gplvm.base_model.kernel(Sample,gplvm.base_model.Xu).detach().numpy().T)
    # B^-1 I
    Noise=np.eye(Q.shape[0])*np.sqrt(noise_var) 
    Noise_term = np.matmul(np.linalg.inv(Noise),np.eye(Q.shape[0]))  # Noise term
    # Knn/ SigmaS
    K_new = np.add(Q,Noise_term) #  Covariance matrix for Y
    # Sigma^-1
    K_inv = np.linalg.inv(K_new)
    temp = np.matmul(np.matmul(Sample.detach().numpy().T,K_inv),Sample.detach().numpy())      
    qx = -0.5*(Q.shape[0]*(D))*(np.log(np.pi*2)) - D/2*np.log(np.linalg.det(K_new)) - 0.5*np.trace(temp)
    return qx  

In [195]:
qx = []
for sample in VS:
    qx.append(Var_Dist(gplvm,sample,D=2,noise_var=0.05))

## PSIS

To compute the PSIS diagnostic, we compute 
$$
r_s = \exp\bigg(\log\bigg(\frac{p(\mathbf{Y}|\mathbf{X}=\mathbf{X}_s)p(\mathbf{X}=\mathbf{X}_s)}{q(\mathbf{X}=\mathbf{X}_s)}\bigg)\bigg),
$$
which leads to
$$
r_s = \exp\big(\log\big(p(\mathbf{Y}|\mathbf{X}=\mathbf{X}_s)p(\mathbf{X}=\mathbf{X}_s)\big)- \log\big(q(\mathbf{X}=\mathbf{X}_s)\big)\big),
$$

In [196]:
Rs = [] # Rations with exp
Rs2 = [] # Rations without exp just log ratios
for i in range(S):
    Rs.append(np.exp(Join[i]-qx[i]))
    Rs2.append(Join[i]-qx[i])

In [197]:
#Rs2

## GPD without smoothed log weights

psislw takes log weights and gives smoothed weights and tail indices.
Passed these log weights in the gpdfitnew to get shape parameter K

In [204]:
print('K:'+ str(gpdfitnew(psislw(np.array(Rs2),Reff=1,overwrite_lw=False)[0])[0]))

K:0.00980392156862745


 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:235: RuntimeWarning:invalid value encountered in log1p
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:251: RuntimeWarning:invalid value encountered in greater_equal
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:271: RuntimeWarning:invalid value encountered in double_scalars


In [203]:
gpdfitnew(np.array(Rs2)) # Unsmoothed weights #second parameter gives the sigma value which is nan 

 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:235: RuntimeWarning:invalid value encountered in log1p
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:240: RuntimeWarning:invalid value encountered in log
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:251: RuntimeWarning:invalid value encountered in greater_equal
 /home/aneeqr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:271: RuntimeWarning:invalid value encountered in double_scalars


(0.00980392156862745, nan)

## Leave One out Predictive Densities

In [200]:
print('K:'+ str(psisloo(np.array(Rs2))[2])) 

K:3.5672771392297467


In [201]:
print('K:'+ str(psisloo(np.array(Rs))[2])) 

K:inf


In [202]:
#psisloo(np.array(Join))

In [205]:
def psisloo(log_lik, **kwargs):
    r"""PSIS leave-one-out log predictive densities.

    Computes the log predictive densities given posterior samples of the log
    likelihood terms :math:`p(y_i|\theta^s)` in input parameter `log_lik`.
    Returns a sum of the leave-one-out log predictive densities `loo`,
    individual leave-one-out log predictive density terms `loos` and an estimate
    of Pareto tail indeces `ks`. The estimates are unreliable if tail index
    ``k > 0.7`` (see more in the references listed in the module docstring).

    Additional keyword arguments are passed to the :meth:`psislw()` function
    (see the corresponding documentation).

    Parameters
    ----------
    log_lik : ndarray
        Array of size n x m containing n posterior samples of the log likelihood
        terms :math:`p(y_i|\theta^s)`.

    Returns
    -------
    loo : scalar
        sum of the leave-one-out log predictive densities

    loos : ndarray
        individual leave-one-out log predictive density terms

    ks : ndarray
        estimated Pareto tail indeces

    """
    # ensure overwrite flag in passed arguments
    kwargs['overwrite_lw'] = True
    # log raw weights from log_lik
    lw = -log_lik
    # compute Pareto smoothed log weights given raw log weights
    lw, ks = psislw(lw, **kwargs)
    # compute
    lw += log_lik
    loos = sumlogs(lw, axis=0)
    loo = loos.sum()
    #print('Ks'+str(ks))
    return loo, loos, ks

def psislw(lw, Reff=1.0, overwrite_lw=False):
    """Pareto smoothed importance sampling (PSIS).

    Parameters
    ----------
    lw : ndarray
        Array of size n x m containing m sets of n log weights. It is also
        possible to provide one dimensional array of length n.

    Reff : scalar, optional
        relative MCMC efficiency ``N_eff / N``

    overwrite_lw : bool, optional
        If True, the input array `lw` is smoothed in-place, assuming the array
        is F-contiguous. By default, a new array is allocated.

    Returns
    -------
    lw_out : ndarray
        smoothed log weights
    kss : ndarray
        Pareto tail indices

    """
    if lw.ndim == 2:
        n, m = lw.shape
    elif lw.ndim == 1:
        n = len(lw)
        m = 1
    else:
        raise ValueError("Argument `lw` must be 1 or 2 dimensional.")
    if n <= 1:
        raise ValueError("More than one log-weight needed.")

    if overwrite_lw and lw.flags.f_contiguous:
        # in-place operation
        lw_out = lw
    else:
        # allocate new array for output
        lw_out = np.copy(lw, order='F')

    # allocate output array for kss
    kss = np.empty(m)
    #print(kss)

    # precalculate constants
    cutoff_ind = - int(np.ceil(min(0.2 * n, 3 * np.sqrt(n / Reff)))) - 1
    cutoffmin = np.log(np.finfo(float).tiny)
    logn = np.log(n)
    k_min = 1/3

    # loop over sets of log weights
    for i, x in enumerate(lw_out.T if lw_out.ndim == 2 else lw_out[None, :]):
        # improve numerical accuracy
        x -= np.max(x)
        # sort the array
        x_sort_ind = np.argsort(x)
        # divide log weights into body and right tail
        xcutoff = max(
            x[x_sort_ind[cutoff_ind]],
            cutoffmin
        )
        expxcutoff = np.exp(xcutoff)
        tailinds, = np.where(x > xcutoff)
        x2 = x[tailinds]
        n2 = len(x2)
        if n2 <= 4:
            # not enough tail samples for gpdfitnew
            k = np.inf
        else:
            # order of tail samples
            x2si = np.argsort(x2)
            # fit generalized Pareto distribution to the right tail samples
            np.exp(x2, out=x2)
            x2 -= expxcutoff
            k, sigma = gpdfitnew(x2, sort=x2si)
        if k >= k_min and not np.isinf(k):
            # no smoothing if short tail or GPD fit failed
            # compute ordered statistic for the fit
            sti = np.arange(0.5, n2)
            sti /= n2
            qq = gpinv(sti, k, sigma)
            qq += expxcutoff
            np.log(qq, out=qq)
            # place the smoothed tail into the output array
            x[tailinds[x2si]] = qq
            # truncate smoothed values to the largest raw weight 0
            x[x > 0] = 0
        # renormalize weights
        x -= sumlogs(x)
        # store tail index k
        kss[i] = k
        #print(k)

    # If the provided input array is one dimensional, return kss as scalar.
    if lw_out.ndim == 1:
        kss = kss[0]

    return lw_out, kss

def sumlogs(x, axis=None, out=None):
    """Sum of vector where numbers are represented by their logarithms.

    Calculates ``np.log(np.sum(np.exp(x), axis=axis))`` in such a fashion that
    it works even when elements have large magnitude.

    """
    maxx = x.max(axis=axis, keepdims=True)
    xnorm = x - maxx
    np.exp(xnorm, out=xnorm)
    out = np.sum(xnorm, axis=axis, out=out)
    if isinstance(out, np.ndarray):
        np.log(out, out=out)
    else:
        out = np.log(out)
    out += np.squeeze(maxx)
    return out
def gpdfitnew(x, sort=True, sort_in_place=False, return_quadrature=False):
    """Estimate the paramaters for the Generalized Pareto Distribution (GPD)

    Returns empirical Bayes estimate for the parameters of the two-parameter
    generalized Parato distribution given the data.

    Parameters
    ----------
    x : ndarray
        One dimensional data array

    sort : bool or ndarray, optional
        If known in advance, one can provide an array of indices that would
        sort the input array `x`. If the input array is already sorted, provide
        False. If True (default behaviour), the array is sorted internally.

    sort_in_place : bool, optional
        If `sort` is True and `sort_in_place` is True, the array is sorted
        in-place (False by default).

    return_quadrature : bool, optional
        If True, quadrature points and weight `ks` and `w` of the marginal posterior distribution of k are also calculated and returned. False by
        default.

    Returns
    -------
    k, sigma : float
        estimated parameter values

    ks, w : ndarray
        Quadrature points and weights of the marginal posterior distribution
        of `k`. Returned only if `return_quadrature` is True.

    Notes
    -----
    This function returns a negative of Zhang and Stephens's k, because it is
    more common parameterisation.

    """
    if x.ndim != 1 or len(x) <= 1:
        raise ValueError("Invalid input array.")

    # check if x should be sorted
    if sort is True:
        if sort_in_place:
            x.sort()
            xsorted = True
        else:
            sort = np.argsort(x)
            xsorted = False
    elif sort is False:
        xsorted = True
    else:
        xsorted = False

    n = len(x)
    PRIOR = 3
    m = 30 + int(np.sqrt(n))

    bs = np.arange(1, m + 1, dtype=float)
    bs -= 0.5
    np.divide(m, bs, out=bs)
    np.sqrt(bs, out=bs)
    np.subtract(1, bs, out=bs)
    if xsorted:
        bs /= PRIOR * x[int(n/4 + 0.5) - 1]
        bs += 1 / x[-1]
    else:
        bs /= PRIOR * x[sort[int(n/4 + 0.5) - 1]]
        bs += 1 / x[sort[-1]]

    ks = np.negative(bs)
    temp = ks[:,None] * x
    np.log1p(temp, out=temp)
    np.mean(temp, axis=1, out=ks)

    L = bs / ks
    np.negative(L, out=L)
    np.log(L, out=L)
    L -= ks
    L -= 1
    L *= n

    temp = L - L[:,None]
    np.exp(temp, out=temp)
    w = np.sum(temp, axis=1)
    np.divide(1, w, out=w)

    # remove negligible weights
    dii = w >= 10 * np.finfo(float).eps
    if not np.all(dii):
        w = w[dii]
        bs = bs[dii]
    # normalise w
    w /= w.sum()

    # posterior mean for b
    b = np.sum(bs * w)
    # Estimate for k, note that we return a negative of Zhang and
    # Stephens's k, because it is more common parameterisation.
    temp = (-b) * x
    np.log1p(temp, out=temp)
    k = np.mean(temp)
    if return_quadrature:
        np.negative(x, out=temp)
        temp = bs[:, None] * temp
        np.log1p(temp, out=temp)
        ks = np.mean(temp, axis=1)
    # estimate for sigma
    sigma = -k / b * n / (n - 0)
    # weakly informative prior for k
    a = 10
    k = k * n / (n+a) + a * 0.5 / (n+a)
    if return_quadrature:
        ks *= n / (n+a)
        ks += a * 0.5 / (n+a)

    if return_quadrature:
        #print('K:'+str(k))
        return k, sigma, ks, w
    else:
        #print('K:'+str(k))
        return k, sigma

def gpinv(p, k, sigma):
    """Inverse Generalised Pareto distribution function."""
    x = np.empty(p.shape)
    x.fill(np.nan)
    if sigma <= 0:
        return x
    ok = (p > 0) & (p < 1)
    if np.all(ok):
        if np.abs(k) < np.finfo(float).eps:
            np.negative(p, out=x)
            np.log1p(x, out=x)
            np.negative(x, out=x)
        else:
            np.negative(p, out=x)
            np.log1p(x, out=x)
            x *= -k
            np.expm1(x, out=x)
            x /= k
        x *= sigma
    else:
        if np.abs(k) < np.finfo(float).eps:
            # x[ok] = - np.log1p(-p[ok])
            temp = p[ok]
            np.negative(temp, out=temp)
            np.log1p(temp, out=temp)
            np.negative(temp, out=temp)
            x[ok] = temp
        else:
            # x[ok] = np.expm1(-k * np.log1p(-p[ok])) / k
            temp = p[ok]
            np.negative(temp, out=temp)
            np.log1p(temp, out=temp)
            temp *= -k
            np.expm1(temp, out=temp)
            temp /= k
            x[ok] = temp
        x *= sigma
        x[p == 0] = 0
        if k >= 0:
            x[p == 1] = np.inf
        else:
            x[p == 1] = -sigma / k
    return x

# KL Divergence

$$
D_\text{KL}(\mathcal{N}_0 \| \mathcal{N}_1) = { 1 \over 2 } \left\{ \operatorname{tr} \left( \boldsymbol\Sigma_1^{-1} \boldsymbol\Sigma_0 \right) + \left( \boldsymbol\mu_1 - \boldsymbol\mu_0\right)^{\rm T} \boldsymbol\Sigma_1^{-1} ( \boldsymbol\mu_1 - \boldsymbol\mu_0 ) - k +\ln { |  \boldsymbol \Sigma_1 | \over | \boldsymbol\Sigma_0 | } \right\},
$$

In [206]:
Sample = VS[0] # Take any random Sample lets take one from our samples.
def Var_Co(gplvm,Sample,noise_var=0.05):
    Q = np.matmul(gplvm.base_model.kernel(Sample,gplvm.base_model.Xu).detach().numpy(),np.linalg.inv(gplvm.base_model.kernel(gplvm.base_model.Xu,gplvm.base_model.Xu).detach().numpy()))
    Q = np.matmul(Q,gplvm.base_model.kernel(Sample,gplvm.base_model.Xu).detach().numpy().T)
    Noise=np.eye(Q.shape[0])*np.sqrt(noise_var) 
    Noise_term = np.matmul(np.linalg.inv(Noise),np.eye(Q.shape[0]))  # Noise term
    K_new = np.add(Q,Noise_term) #  Covariance matrix for Y
    return K_new

def Pos_Co(Y,Xs,N,D,Q,kernel,noise_var=0.05):
    K= kernel.K(np.array(Xs),np.array(Xs)) # Covariance matrix for K(Xs,Xs), KNN
    Noise=np.eye(K.shape[0])*np.sqrt(noise_var)  # B^-1 Noise vector
    Noise_term = np.matmul(np.linalg.inv(Noise),np.eye(K.shape[0]))  # B^-1 I
    K_new = np.add(K,Noise_term) #  KNN + B^(-1)I_{N}
    return K_new



In [207]:
K_var = Var_Co(gplvm,Sample)
K_Pos = Pos_Co(Y=Y,Xs=Sample.detach().numpy(),N=150,D=4,Q=2,kernel=kernel)

In [208]:
def KL_Diverg(mean1,mean2,K_var,K_Pos):
    mid_term = np.subtract(mean1,mean2)
    K_inv = np.linalg.inv(K_Pos) # Matrix 
    print(np.trace(np.matmul(K_inv,K_var)))
    print(np.matmul(np.matmul(mid_term.T,K_inv),mid_term))
    print(np.log(np.divide(np.linalg.det(K_Pos),np.linalg.det(K_var))))
    return 0.5
    

In [209]:
KL_Diverg(np.array(m.X.mean),gplvm.get_param('X_loc').detach().numpy(),K_var,K_Pos)

154.62632288988706
[[16.67081723  3.26879737]
 [ 3.26879737 33.17586676]]
-1.9054271744715265


0.5

In [210]:
Rs2

[-925.2973315541112,
 -932.0725197287236,
 -940.7963037223631,
 -945.3842529675011,
 -935.903447802732,
 -945.5582718762436,
 -936.5838106227084,
 -958.2299801687036,
 -944.193407649369,
 -942.4414291606791,
 -945.4721737966797,
 -955.4324014171677,
 -947.9025538320938,
 -954.3556306827629,
 -951.3335378073441,
 -935.2073013339794,
 -935.1000973822136,
 -964.7890849831392,
 -957.7606323533887,
 -945.0188094039634,
 -928.0878157685709,
 -944.5483148684995,
 -949.2980239904329,
 -946.6947793912094,
 -952.2968603879907,
 -942.9422673073642,
 -931.3150061685176,
 -947.3298264872111,
 -958.3790150060113,
 -951.0933767710188,
 -946.725295166102,
 -937.3287043069136,
 -940.4527983373575,
 -946.9621718226612,
 -962.2260664341122,
 -940.0093744040944,
 -942.9003340160855,
 -941.1736998256686,
 -951.972667132275,
 -939.3235549879613,
 -934.4579195627168,
 -944.8126799646169,
 -948.0063838186793,
 -951.0411237418028,
 -962.9065750842063,
 -961.9851863285793,
 -950.1013060859344,
 -948.37703109671